In [1]:
import torch
import torch.nn as nn
import torch.optim as optimize
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import pandas as pd
from skimage import io
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import classification_report

In [2]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs, L1RegWeight):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        torch.cuda.empty_cache()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            batches = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                batches += 1
                print("Batch " + str(batches))
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    if (phase != 'val' and L1RegWeight != 0):
                        loss += L1RegWeight * (sum(p.abs().sum() for p in model.parameters()))

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.detach().item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).detach()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [3]:
def doValidation(model, dataloader):
    results = {"real": [], "pred": []}
    model.eval()
    torch.set_grad_enabled(False)
    for inputs, labels in dataloader:
        for x in labels.tolist():
            results['real'].append(x)
        inputs = inputs.to(device)
        output = model(inputs)
        for x in torch.argmax(output, dim=1).tolist():
            results["pred"].append(x)
    return results

In [4]:
def initModel(num_classes, batchNormalization):
    if batchNormalization:
        model_ft = models.vgg16_bn(pretrained=True)
    else:
        model_ft = models.vgg16(pretrained=True)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
    return model_ft

In [5]:
modelTest = initModel(3, True)

In [6]:
batch_size = 128
lr = 0.001
momentum = 0.9
regL2_mult = 0.0005

In [7]:
criterion = nn.CrossEntropyLoss()
optim = optimize.SGD(modelTest.parameters(), lr=lr, momentum=momentum, weight_decay=regL2_mult)

In [8]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomCrop(224),
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor()
    ]),
}

In [9]:
class DisastersDatasets(Dataset):
    def __init__(self, csv, root_dir, transform=None):
        self.csv = pd.read_csv(csv, delim_whitespace=True, header=None, usecols=[0,1], names=["img","lbl"])
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.csv["img"][idx])

        image = io.imread(img_name)
        image = Image.fromarray(image)
        
        if self.transform:
            image = self.transform(image)
        label = self.csv["lbl"][idx]

        if (image.shape[0] == 4):
            image = image[0:3]

        return image, label

In [10]:
trainSet = DisastersDatasets("ASONAM17_Damage_Image_Dataset\\ruby.train", "ASONAM17_Damage_Image_Dataset", data_transforms['train'])
testSet = DisastersDatasets("ASONAM17_Damage_Image_Dataset\\ruby.test", "ASONAM17_Damage_Image_Dataset", data_transforms['val'])
devSet = DisastersDatasets("ASONAM17_Damage_Image_Dataset\\ruby.dev", "ASONAM17_Damage_Image_Dataset", data_transforms['val'])

In [1]:
train_dataloader = DataLoader(trainSet, batch_size = 14, shuffle=True)
test_dataloader = DataLoader(testSet, batch_size = 14, shuffle=True)
dev_dataloader = DataLoader(devSet, batch_size = 14, shuffle=True)

NameError: name 'DataLoader' is not defined

In [12]:
device = torch.device("cuda:0")
modelTest.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [13]:
torch.cuda.empty_cache()

In [33]:
modelTest, history = train_model(modelTest, {"train": train_dataloader, "val": test_dataloader}, criterion, optim, 3, 0)

Epoch 0/2
----------
Batch 1
Batch 2


RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 6.00 GiB total capacity; 3.96 GiB already allocated; 130.50 MiB free; 4.37 GiB reserved in total by PyTorch)

In [15]:
torch.cuda.empty_cache()

In [16]:
lr = 0.0001
optim = optim = optimize.SGD(modelTest.parameters(), lr=lr, momentum=momentum, weight_decay=regL2_mult)
modelTest, historyTwo = train_model(modelTest, {"train": train_dataloader, "val": test_dataloader}, criterion, optim, 3, 0)
history.append(historyTwo)

Epoch 0/2
----------
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19
Batch 20
Batch 21
Batch 22
Batch 23
Batch 24
Batch 25
Batch 26
Batch 27
Batch 28
Batch 29
Batch 30
Batch 31
Batch 32
Batch 33
Batch 34
Batch 35
Batch 36
Batch 37
Batch 38
Batch 39
Batch 40
Batch 41
Batch 42
Batch 43
Batch 44
Batch 45
Batch 46
Batch 47
Batch 48
Batch 49
Batch 50
Batch 51
Batch 52
Batch 53
Batch 54
Batch 55
Batch 56
Batch 57
Batch 58
Batch 59
Batch 60
Batch 61
Batch 62
Batch 63
Batch 64
Batch 65
Batch 66
Batch 67
Batch 68
Batch 69
Batch 70
Batch 71
Batch 72
Batch 73
Batch 74
Batch 75
Batch 76
Batch 77
Batch 78
Batch 79
Batch 80
Batch 81
Batch 82
Batch 83
Batch 84
Batch 85
Batch 86
Batch 87
Batch 88
Batch 89
Batch 90
Batch 91
Batch 92
Batch 93
Batch 94
Batch 95
Batch 96
Batch 97
Batch 98
train Loss: 0.2047 Acc: 0.9291
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Bat

In [17]:
torch.cuda.empty_cache()

In [18]:
lr = 0.00001
optim = optim = optimize.SGD(modelTest.parameters(), lr=lr, momentum=momentum, weight_decay=regL2_mult)
modelTest, historyThree = train_model(modelTest, {"train": train_dataloader, "val": test_dataloader}, criterion, optim, 3, 0)
history.append(historyThree)

Epoch 0/2
----------
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19
Batch 20
Batch 21
Batch 22
Batch 23
Batch 24
Batch 25
Batch 26
Batch 27
Batch 28
Batch 29
Batch 30
Batch 31
Batch 32
Batch 33
Batch 34
Batch 35
Batch 36
Batch 37
Batch 38
Batch 39
Batch 40
Batch 41
Batch 42
Batch 43
Batch 44
Batch 45
Batch 46
Batch 47
Batch 48
Batch 49
Batch 50
Batch 51
Batch 52
Batch 53
Batch 54
Batch 55
Batch 56
Batch 57
Batch 58
Batch 59
Batch 60
Batch 61
Batch 62
Batch 63
Batch 64
Batch 65
Batch 66
Batch 67
Batch 68
Batch 69
Batch 70
Batch 71
Batch 72
Batch 73
Batch 74
Batch 75
Batch 76
Batch 77
Batch 78
Batch 79
Batch 80
Batch 81
Batch 82
Batch 83
Batch 84
Batch 85
Batch 86
Batch 87
Batch 88
Batch 89
Batch 90
Batch 91
Batch 92
Batch 93
Batch 94
Batch 95
Batch 96
Batch 97
Batch 98
train Loss: 0.1724 Acc: 0.9342
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Bat

In [19]:
results = doValidation(modelTest, dev_dataloader)
targetNames = ['none', 'mild', 'severe']
print(results['real'])
print(results['pred'])
print(classification_report(results['real'], results['pred'], target_names=targetNames))

[0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 2, 2, 0, 0, 0, 2, 0, 2, 2, 0, 1, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 1, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 1, 2, 2, 2, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 1, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 1, 2, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 2, 0, 2, 2, 1, 2, 2, 2, 2, 1, 0, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 2, 1, 2, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 2, 2, 1, 2, 0, 2, 1, 2, 0, 1, 2, 0, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 0, 1, 1, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 0, 0, 0, 1, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 1, 2, 1, 0, 